In [8]:
# 画骑手轨迹，用于Return Path的motivation
# Yi Ding
# 07/17/18

import matplotlib.pyplot as plt
import gmplot
import pandas as pd
import os

In [63]:
# Path Assign
data_path = '/Users/eleme-yi/Documents/PhD/GitHub Pages/dymodi.github.io/Research/Paper/Rider-Routing/data/'
figure_path = '/Users/eleme-yi/Documents/PhD/GitHub Pages/dymodi.github.io/Research/Paper/Rider-Routing/figures/'



# Read in trace data
trace_file = '.'.join(['_'.join(['rider_trace',date_str, str(rider_id)]),'xlsx'])
df_rider_trace = pd.read_excel(os.path.join(data_path, trace_file))
print(df_rider_trace.head())



    taker_id            record_time  time_stamp   longitude   latitude  \
0  113049002  2018-07-18 08:14:25.0  1531872865  123.416660  41.811597   
1  113049002  2018-07-18 08:17:26.0  1531873046  123.413226  41.809427   
2  113049002  2018-07-18 08:20:25.0  1531873225  123.413226  41.809427   
3  113049002  2018-07-18 08:23:25.0  1531873405  123.413226  41.809427   
4  113049002  2018-07-18 08:26:25.0  1531873585  123.413226  41.809427   

                                              detail  zone_id  city_id  \
0  {"speed": -1, "course": -1, "altitude": 59, "l...        0        0   
1  {"speed": 0.1, "course": -1, "altitude": 55.8,...        0        0   
2  {"speed": 0.1, "course": -1, "altitude": 55.8,...        0        0   
3  {"speed": 0.1, "course": -1, "altitude": 55.8,...        0        0   
4  {"speed": 0.1, "course": -1, "altitude": 55.8,...        0        0   

           dt          type  
0  2018-07-18  TALARIS_TEAM  
1  2018-07-18  TALARIS_TEAM  
2  2018-07-18  TALAR

In [72]:
# Here we def the function to draw tracking event
# date_str = '2018-07-18'
# rider_id_list = [106279218]
def plot_tracking_event(date_str, rider_id_list, data_path, need_trace = False):
    
    # Plot size
    waypoint_size = 200
    
    # Read in file as pandas dataframe
    # Read in tracking event data
    track_event_file = '_'.join(['tracking_event',date_str])
    track_event_file = '.'.join([track_event_file,'xlsx'])
    df_tracking_event_all = pd.read_excel(os.path.join(data_path, track_event_file))
    # print(df_tracking_event_all.head())
    
    for rider_id in rider_id_list:
        print('plot tracking event for rider_id:',rider_id)
        index_rider = df_tracking_event_all['carrier_driver_id'] == rider_id
        df_tracking_event = df_tracking_event_all[index_rider]
        # print(df_tracking_event.head())
        index_normal = df_tracking_event['is_normal'] == 1
        index_return = df_tracking_event['is_normal'] == 0
        df_normal_order = df_tracking_event[index_normal]
        df_return_order = df_tracking_event[index_return]
        index_normal_pickup = df_normal_order['shipping_state'] == 80
        index_normal_delivr = df_normal_order['shipping_state'] == 40
        df_normal_pickup = df_normal_order[index_normal_pickup]
        df_normal_delivr = df_normal_order[index_normal_delivr]
        index_return_pickup = df_return_order['shipping_state'] == 80
        index_return_delivr = df_return_order['shipping_state'] == 40
        df_return_pickup = df_return_order[index_return_pickup]
        df_return_delivr = df_return_order[index_return_delivr]
        # print(df_normal_order.head())
        # print(df_normal_pickup.head())
        # print(df_return_pickup.head())
        # Plot
        file_name = '_'.join(['rider_tracking_event',date_str, str(rider_id)])
        file_name = '.'.join([file_name,'html'])
        figure_name = os.path.join(figure_path, file_name)
        if df_normal_pickup.empty:
            lat_cen = df_return_pickup['latitude'].iloc[0]
            lon_cen = df_return_pickup['longitude'].iloc[0]
        else:
            lat_cen = df_normal_pickup['latitude'].iloc[0]
            lon_cen = df_normal_pickup['longitude'].iloc[0]
        # print('lat/lon cen:',lat_cen, lon_cen)
        gmap = gmplot.GoogleMapPlotter(lat_cen, lon_cen, 16)
        gmap.scatter(df_normal_pickup['latitude'].tolist(), df_normal_pickup['longitude'].tolist(), 
                     'salmon', size = waypoint_size, marker=True)
        gmap.scatter(df_normal_delivr['latitude'].tolist(), df_normal_delivr['longitude'].tolist(), 
                     'yellowgreen', size = waypoint_size, marker=True)
        gmap.scatter(df_return_pickup['latitude'].tolist(), df_return_pickup['longitude'].tolist(), 
                     'crimson', size = waypoint_size, marker=True)
        gmap.scatter(df_return_delivr['latitude'].tolist(), df_return_delivr['longitude'].tolist(), 
                     'green', size = waypoint_size, marker=True)
        if need_trace:
            # Read in trace data
            trace_file = '.'.join(['_'.join(['rider_trace',date_str, str(rider_id)]),'xlsx'])
            df_rider_trace = pd.read_excel(os.path.join(data_path, trace_file))
            gmap = gmplot.GoogleMapPlotter(df_rider_trace['latitude'].iloc[0], df_rider_trace['longitude'].iloc[0], 16)
            gmap.plot(df_rider_trace['latitude'].tolist(), df_rider_trace['longitude'].tolist(), 'blue', edge_width=10)
            # gmap.scatter(df_rider_trace['latitude'].tolist(), df_rider_trace['longitude'].tolist(), 'green', marker=True)        
        gmap.draw(figure_name)
    

In [65]:
# Plot trace
date_str = '2018-07-18'
rider_id = 113049002

figure_file = '.'.join(['_'.join(['rider_trace', date_str, str(rider_id)]),'html'])
figure_name = os.path.join(figure_path, figure_file)

gmap = gmplot.GoogleMapPlotter(df_rider_trace['latitude'].iloc[0], df_rider_trace['longitude'].iloc[0], 16)
gmap.plot(df_rider_trace['latitude'].tolist(), df_rider_trace['longitude'].tolist(), 'blue', edge_width=10)
# gmap.scatter(df_rider_trace['latitude'].tolist(), df_rider_trace['longitude'].tolist(), 'green', marker=True)
gmap.draw(figure_name)

In [74]:
# Plot the event and trace for single rider for slide
date_str = '2018-07-18'
rider_id_list = [113049002]
plot_tracking_event(date_str, rider_id_list, data_path, need_trace = True)

plot tracking event for rider_id: 113049002


In [69]:
# Plot pickup point
# Prepare data
rider_id_list = df_tracking_event_all['carrier_driver_id'].drop_duplicates().tolist()
print('rider_list:',rider_id_list)

# Date
date_str = '2018-07-18'
# 
plot_tracking_event(date_str, rider_id_list, data_path)

    

rider_list: [1042584, 109863922, 4396890, 836222, 112823786, 10799990, 113049002, 110934626, 102383762, 106279218]
plot tracking event for rider_id: 1042584
plot tracking event for rider_id: 109863922
plot tracking event for rider_id: 4396890
plot tracking event for rider_id: 836222
plot tracking event for rider_id: 112823786
plot tracking event for rider_id: 10799990
plot tracking event for rider_id: 113049002
plot tracking event for rider_id: 110934626
plot tracking event for rider_id: 102383762
plot tracking event for rider_id: 106279218
